# Object Detection

Object detection in images using a pretrained YoloV3 model and OpenCV.

For YoloV3 model: [Darknet YoloV3](https://pjreddie.com/darknet/yolo/)

For identifiable object classes: [Coco](https://github.com/pjreddie/darknet/blob/master/data/coco.names)

### Import Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Load YoloV3 model and Image

In [2]:
def modelLoading():
    # reads neural network model
    net = cv2.dnn.readNet("models\yolov3.weights", "models\yolov3.cfg")
    
    # reads all identifiable objects from coco.names    
    with open("models\coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # getting indexes of all output layers
    layers_names = net.getLayerNames()
    out_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    
    # returns neural network, object classes, output_layers of network
    return net, classes, out_layers

def imageProccesing(imgPath):
    # reads and resizes image using scale factor of 0.7
    img = cv2.imread(imgPath)
    img = cv2.resize(img, None, fx=0.7, fy=0.7)
    height, width, channels = img.shape
    
    # returns image, height, width, channel(RBG = 3)
    return img, height, width, channels

### Detect objects

In [3]:
def objectDetection(img, net, outputLayers):
    # detects potential objects in image
    blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
    # input detected potential objects in neural network
    net.setInput(blob)
    # get output values from network
    outputs = net.forward(outputLayers)
    # return output ids of objects 
    return outputs

### Bounding box and label objects

In [4]:
def boundingBox(img, outputs, height, width, classes):
    boxes, confs, class_ids = [], [], []
    # colors for boxes in BGR
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    
    # finding boxes(box dimensions), confidence levels, class_ids(object ids)
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > 0.3:
                cx = int(detect[0] * width)
                cy = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)
                x = int(cx - w/2)
                y = int(cy - h / 2)
                boxes.append([x, y, w, h])
                confs.append(float(conf))
                class_ids.append(class_id)

    # perform non max suppression on boxes and confidence levels
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    
    # creating boxes
    for i in range(len(boxes)):
        if i in indexes:
            # finds label for object from classes using id
            label = str(classes[class_ids[i]])
            # select box color
            color = colors[i]
            # dimensions of object box
            x, y, w, h = boxes[i]
            # create box
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            # label object box
            cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_PLAIN, 1, color, 1)
            
    # plt.imshow(img[:,:,::-1])
    return img

### Combine all functions for detection

In [5]:
def image_detection(path):
    model, classes, output_layers = modelLoading() # load yolo model
    image, height, width, channels = imageProccesing(path) # read and process image
    outputs = objectDetection(image, model, output_layers) # detect objects in image
    labelled_img = boundingBox(image, outputs, height, width, classes) # create bounding box and label objects
    cv2.imshow("Objects detected in image", labelled_img)
    cv2.waitKey(0)

### Running Code

In [6]:
# to run object detection pass the image path to the function
image_detection("images/work.jpg")